# Conversion to NbDev

A short description of Converting HighResAnalysis to NbDev

## Motivation

HighResAnalysis package was developed for analysis of the data that a diamond sensor group at ETH IPA is using to study the data collected in 2018 and 2019 beam tests at CERN and DESY. The main focus of study was on two 3D diamond detectors. The package works and efficient, however, the documentation is sparse and the future development and maintenance of the code is difficult. By converting the code to a notebook style will hopefully improve that.

## lib2nbdev

lib2nbdev is a nice little package originally created by [Zach Mueller at Novetta](https://www.novetta.com/2021/06/lib2nbdev/). The package was however based on the old version of the NbDev so I started by converting it to the new version. I hopefully will describe the details of that conversion on the projects [website](https://dmitryhits.github.io/lib2nbdev/)

## Conversion

### First steps

I forked the project from [https://github.com/diamondIPP/HighResAnalysis](https://github.com/diamondIPP/HighResAnalysis) to my personal account and cloned it on my laptop. The [lib2nbdev2](https://pypi.org/project/lib2nbdev2/) package provides a `convert_lib` command that, when run in the project directory, creates a notebook for every `.py` file in the `libdir`. The `libdir` is determined from the `settings.ini` file, which is automatically created with most of the information inferred from the github info of the repository. Whatever the program is not able to infer it will ask.

::: {.callout-note}
All of your `.py` files should be preferably inside one directory (`libdir`) which is inside your root github project directory.
:::

My project did not have such directory, so I created it and moved all the dirs with `.py` files into it.

This is however all the `convert_lib` program does at the moment. Most of the next steps are done manually.
First, I run [`nbdev_new`](https://nbdev.fast.ai/api/cli.html#nbdev_new), which copied an extra necessary files into my project, some of which, for example, `00_core.ipynb` were not needed, so I removed them.

### Debugging

The following steps involved long process of adding basic, not yet extensive, documentation, because the `convert_lib` created only standard default titles and descriptions (see below), which need to be changed in order to have at least a useful sidebar.

![](default_first_cells.png)

Simultaneously, I was running the notebooks and fixing the errors that come up and also placing

```python
#| hide
from nbdev import *
nbdev_export()
```

At the bottom of each notebook

:::{.callout-tip collapse="true"}
## Note to Self
I should add that to `convert_lib`
:::

First, errors I fixed were the import errors. In the notebooks I needed a path to the project that starts from the project lib dir.
```python
import HighResAnalysis.convert
```
instead of just:
```python
import convert
```